In [ ]:
!pip install datasets

In [145]:
import pandas as pd
from datasets import load_from_disk, Dataset, DatasetDict
import re
import random

In [146]:
# pandas datasframe으로 변환 및 필요한 컬럼만 추출하는 함수
def dataset_to_df(dataset):
  data_list = []

  for data in dataset:
    data_list.append({
        'title' : data["title"],
        "context" : data["context"],
        "question" : data["question"],
        "id" : data["id"],
        "answer_start" : data["answers"]["answer_start"][0],
        "answer_text" : data["answers"]["text"][0],
        "document_id" : data["document_id"]
    })
  df = pd.DataFrame(data_list)
  return df

In [147]:
path = "your_train_dataset_path"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])

In [148]:
print(f"train dataset의 context의 타입 : {type(dataset['train']['context'])}")
print(f"train dataset의 context의 길이 : {len(dataset['train']['context'])}")

train dataset의 context의 타입 : <class 'list'>
train dataset의 context의 길이 : 3952


In [149]:
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [150]:
# 괄호 안에 있는 단어중 한글이나 숫자가 아닌 것들만 추출하는 함수
def get_parentheses_value(df, column):
  values_in_parentheses = []
  for idx, data in enumerate(df[column]):
    for match in re.finditer(r'\((.*?)\)', data):
      value = match.group(1)
      start_pos = match.start(1)

      if not re.search(r'[가-힣0-9]', value):
                values_in_parentheses.append({
                    'value': value,
                    'start_pos': start_pos,
                    'length' : len(value)
                })

  return values_in_parentheses

In [151]:
# 랜덤 샘플링
sample_train_df = train_df.sample(n=100, random_state=42).reset_index(drop=True)

In [152]:
sample_train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,윤치호,독립신문\n독립신문의 주필과 발행인을 하던 그는 신문사를 떠맡게 되어 1898년 독...,윤직선의 아들 이름은?,1764,윤극영,5293
1,화흡,"젊어서는 허자장, 번자소(樊子昭)와 함께 명성이 있었다. 후한 말, 효렴으로 천거되...",유종이 조조에게 투항한 해는?,236,건안 13년,27275
2,김집 (1574년),이괄의 난\n1623년(광해군 15년) 인조반정 직후 학행으로 천거되어 사헌부대사헌...,김집이 몸이 아파서 내려놓은 관직은?,187,부여현감(扶餘縣監),14683
3,스웨덴의 통일,"또는 통일이 한 명의 왕 치세에 불연속적으로 이루어진 것이 아니라, 점진적 과정으로...",중세 초기의 스웨덴에서 역할을 제대로 수행하지 못한 것은?,1086,왕권,57654
4,대한불교조계종,대한불교조계종(大韓佛敎曹溪宗)은 대한민국 불교 최대 종단(종파)이다. 조계종은 중국...,돈오점수와 혜능의 법맥은 양립할 수 없다고 주장한 사람은?,267,성철,5568


In [153]:
print(get_parentheses_value(sample_train_df, "context"))

[{'value': '反', 'start_pos': 663, 'length': 1}, {'value': '高永根', 'start_pos': 891, 'length': 3}, {'value': '尹稷善', 'start_pos': 1738, 'length': 3}, {'value': '樊子昭', 'start_pos': 14, 'length': 3}, {'value': '簡侯', 'start_pos': 867, 'length': 2}, {'value': '和离', 'start_pos': 882, 'length': 2}, {'value': '扶餘縣監', 'start_pos': 192, 'length': 4}, {'value': '臨陂縣令', 'start_pos': 568, 'length': 4}, {'value': '世子翊衛司衛率', 'start_pos': 829, 'length': 7}, {'value': '全羅都事', 'start_pos': 844, 'length': 4}, {'value': 'state', 'start_pos': 276, 'length': 5}, {'value': 'statehood', 'start_pos': 414, 'length': 9}, {'value': '大韓佛敎曹溪宗', 'start_pos': 8, 'length': 7}, {'value': 'pappas', 'start_pos': 199, 'length': 6}, {'value': 'Dia di San Juan', 'start_pos': 298, 'length': 15}, {'value': '齒苔, dental plaque', 'start_pos': 3, 'length': 17}, {'value': '糖', 'start_pos': 311, 'length': 1}, {'value': 'Alvini', 'start_pos': 320, 'length': 6}, {'value': 'Tegetthoff', 'start_pos': 456, 'length': 10}, {'value': 'The Tr

In [154]:
sample_train_df.loc[0]["context"]

"독립신문\\n독립신문의 주필과 발행인을 하던 그는 신문사를 떠맡게 되어 1898년 독립신문사 제2대 사장이 되었으나, 관직에 임용되면서 사퇴하였고, 1899년 이후 독립협회에 대한 탄압은 가중되었다. 윤치호는 독립협회의 탄압·해산 시 외국인의 집에 은신하고 있다가 1899년 1월 7일자로 덕원감리사 겸 덕원부윤에 임명되었고, 윤치호는 1899년 2월 2일 이를 수락하였다. 그는 1898년 관직 생활을 하면서도 배재학당에 나가 학생들에게 토론하는 방법을 가르쳤다. 그는 토론회의 소모임을 1899년 2월까지 맡아보았는데, 학생들이 토론에서 패했을 때 감정적으로 발언하는 것을 통제하고 억제하면서, 감정을 조절하고 대화, 토론하는 방법을 훈련시켰다.\\n\\n후에 숭실대학교 사학과 교수 유영렬은 '민중운동의 최고 지도자였던 윤치호에 대한 이같은 조처는 당시 법부대신 윤웅렬의 노력과 윤치호에 대한 고종의 친애감, 그리고 평소 대인관계가 원만했던 윤치호와 일부 관료들과의 친분관계가 크게 작용했던 것'으로 보았다 한편 윤치호 자신이 개혁 운동을 포기하고 타협한 것에 대하여 훗날 숭실대학교 사학과 교 수 유영렬은 '윤치호에 대한 감리사직의 임명은 (중앙 정계로부터의) 일종의 회유적 추방이었으며, 윤치호의 감리사직 수락은 자신과 가족의 안전을 위한 일종의 자구책이었던 '이라고 보았다. 반(反)정부 민권 운동의 최고지도자가, 그것도 민중 지도자들이 대거 체포 구금되는 상황에서, 극복의 대상인 수구반동하의 지방관직을 수락한 사실은 일종의 변절적 자세로 보지 않을 수 없으며, 전통적 통치체제에 대한 비판의식의 불철저성을 지적하지 않을 수 없다. 고 해석하였다.\\n\\n1899년 1월 그에게 함경남도 원산부윤으로 임명되리라는 설이 돌았다. 1월 16일 오후 7시경 은밀히 일본인 집에 피신한 고영근(高永根)의 행방을 알고 그를 찾아갔다. 윤치호를 만났던 고영근은 그에게 원산부윤직에 나갈 것이냐고 물었고, 윤치호가 대답을 주저하자 그의 아버지 윤웅렬이 법부대신으로 승진한 것은 만민공

# 다른 언어를 없애게 되면 answer_start의 값도 달라져야 하지 않을까?
- 방법1) start_pos가 answer_start보다 작으면 answer_start에서 사라지는 값의 길이 만큼 빼기
- 방법2) 예) context = “맥북은 애플에서 만들었다. 아이폰은 애플에서 만들었다. 애플의 유명한 인물은 스티븐 잡스이다.” 면 question이 “맥북은 어디에서 만들어졌나”면 “애플”이다. 그러면 여기서 answer_start가 4번이다. 하지만 개행 전처리하면 바뀔 수도 있다. 그래서 정답이라는 애플의 index을 다 구한다. 그리고 n번째가 answer_start이면 전처리 후에도 n번째의 index가 answer_start가 될 수 있다.

In [155]:
sample_train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,윤치호,독립신문\n독립신문의 주필과 발행인을 하던 그는 신문사를 떠맡게 되어 1898년 독...,윤직선의 아들 이름은?,1764,윤극영,5293
1,화흡,"젊어서는 허자장, 번자소(樊子昭)와 함께 명성이 있었다. 후한 말, 효렴으로 천거되...",유종이 조조에게 투항한 해는?,236,건안 13년,27275
2,김집 (1574년),이괄의 난\n1623년(광해군 15년) 인조반정 직후 학행으로 천거되어 사헌부대사헌...,김집이 몸이 아파서 내려놓은 관직은?,187,부여현감(扶餘縣監),14683
3,스웨덴의 통일,"또는 통일이 한 명의 왕 치세에 불연속적으로 이루어진 것이 아니라, 점진적 과정으로...",중세 초기의 스웨덴에서 역할을 제대로 수행하지 못한 것은?,1086,왕권,57654
4,대한불교조계종,대한불교조계종(大韓佛敎曹溪宗)은 대한민국 불교 최대 종단(종파)이다. 조계종은 중국...,돈오점수와 혜능의 법맥은 양립할 수 없다고 주장한 사람은?,267,성철,5568


In [156]:
# answer_text와 같은 단어들의 시작 인덱스를 구하는 코드
def get_answer_word_idx(context, answer):
  indices = [m.start() for m in re.finditer(answer, context)]
  return indices

In [157]:
sample_train_df["answer_word_idx"] = sample_train_df.apply(lambda row: get_answer_word_idx(row["context"], row["answer_text"]), axis=1)

In [158]:
sample_train_df.head()

,title,context,question,answer_start,answer_text,document_id,answer_word_idx
0,윤치호,독립신문\n독립신문의 주필과 발행인을 하던 그는 신문사를 떠맡게 되어 1898년 독...,윤직선의 아들 이름은?,1764,윤극영,5293,"[1764, 1807]"
1,화흡,"젊어서는 허자장, 번자소(樊子昭)와 함께 명성이 있었다. 후한 말, 효렴으로 천거되...",유종이 조조에게 투항한 해는?,236,건안 13년,27275,[236]
2,김집 (1574년),이괄의 난\n1623년(광해군 15년) 인조반정 직후 학행으로 천거되어 사헌부대사헌...,김집이 몸이 아파서 내려놓은 관직은?,187,부여현감(扶餘縣監),14683,[]
3,스웨덴의 통일,"또는 통일이 한 명의 왕 치세에 불연속적으로 이루어진 것이 아니라, 점진적 과정으로...",중세 초기의 스웨덴에서 역할을 제대로 수행하지 못한 것은?,1086,왕권,57654,"[740, 771, 1086, 1134]"
4,대한불교조계종,대한불교조계종(大韓佛敎曹溪宗)은 대한민국 불교 최대 종단(종파)이다. 조계종은 중국...,돈오점수와 혜능의 법맥은 양립할 수 없다고 주장한 사람은?,267,성철,5568,[267]


In [159]:
# 생성한 answer_word_idx에서 answer_start와 같은 값을 가지는 인덱스 번호 저장
def get_answer_idx(answer_start, answer_word_idx):
  re_idx = 0
  for idx, data in enumerate(answer_word_idx):
    if data == answer_start:
      re_idx = int(idx)

  return re_idx

In [160]:
sample_train_df["answer_word_true_idx"] = sample_train_df.apply(lambda row: get_answer_idx(row["answer_start"], row["answer_word_idx"]), axis=1)

In [161]:
sample_train_df.head()

,title,context,question,answer_start,answer_text,document_id,answer_word_idx,answer_word_true_idx
0,윤치호,독립신문\n독립신문의 주필과 발행인을 하던 그는 신문사를 떠맡게 되어 1898년 독...,윤직선의 아들 이름은?,1764,윤극영,5293,"[1764, 1807]",0
1,화흡,"젊어서는 허자장, 번자소(樊子昭)와 함께 명성이 있었다. 후한 말, 효렴으로 천거되...",유종이 조조에게 투항한 해는?,236,건안 13년,27275,[236],0
2,김집 (1574년),이괄의 난\n1623년(광해군 15년) 인조반정 직후 학행으로 천거되어 사헌부대사헌...,김집이 몸이 아파서 내려놓은 관직은?,187,부여현감(扶餘縣監),14683,[],0
3,스웨덴의 통일,"또는 통일이 한 명의 왕 치세에 불연속적으로 이루어진 것이 아니라, 점진적 과정으로...",중세 초기의 스웨덴에서 역할을 제대로 수행하지 못한 것은?,1086,왕권,57654,"[740, 771, 1086, 1134]",2
4,대한불교조계종,대한불교조계종(大韓佛敎曹溪宗)은 대한민국 불교 최대 종단(종파)이다. 조계종은 중국...,돈오점수와 혜능의 법맥은 양립할 수 없다고 주장한 사람은?,267,성철,5568,[267],0


In [ ]:
for i in range(len(sample_train_df)):
  if len(sample_train_df.loc[i]["answer_word_idx"]) == 0:
    continue
  print(f"{i} = ", end=" ")
  if sample_train_df.loc[i]["answer_start"] == sample_train_df.loc[i]["answer_word_idx"][sample_train_df.loc[i]["answer_word_true_idx"]]:
    print("True")
  else:
    print("False")

# 외래어 제거 + answer_start 값 변경

In [163]:
# 괄호 안에 있는 단어중 한글이나 숫자가 아닌 것들만 추출하는 함수
def get_parentheses_value(df, column):
  values_in_parentheses = []
  for idx, data in enumerate(df[column]):
    for match in re.finditer(r'\((.*?)\)', data):
      value = match.group(1)
      start_pos = match.start(1)

      if not re.search(r'[가-힣0-9]', value):
                values_in_parentheses.append({
                    'value': value,
                    'start_pos': start_pos,
                    'length' : len(value)
                })

  return values_in_parentheses

In [164]:
# 랜덤 샘플링
sample_train_df = train_df.sample(n=100, random_state=11).reset_index(drop=True)

In [165]:
sample_train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,로버트 라이트,"라이트는 종교 주제, 특히 '신'이라는 개념의 역사와 진화에 관해 광범위하게 글을 ...",대중을 통제하기 위해 위정자들의 수단이 되기도 한 것은?,387,종교,57743
1,하프라이프의 등장인물 목록,"일라이 밴스(Eli Vance)는 하프라이프와 하프라이프 2에 등장하는 캐릭터로, ...",고든은 블랙메사 연구소의 어디에서 일했나요?,287,C구역,24718
2,튀코 브라헤,"튀코 브라헤(Tycho Brahe, 1546년 12월 14일 ~ 1601년 10월 ...",튀코 브라헤는 프레데리크 2세가 사망한 후 어디로 이주하나요?,307,프라하,4944
3,동적 평형 상태,"소다로 찬 새로운 병에서, 수용액 상태의 이산화 탄소 농도는 특정 값을 가진다. 만...",농도산수가 일정하기 위한 전제조건은?,589,평형상태,6502
4,듀이,1938년 듀이는 인기있는 민주당 현직 주지사 허버트 리먼을 상대로 뉴욕 주지사를 ...,듀이가 첫 번째로 뉴욕 주지사가 된 해는?,233,1942년,22802


In [166]:
sample_train_df["answer_word_idx"] = sample_train_df.apply(lambda row: get_answer_word_idx(row["context"], row["answer_text"]), axis=1)

In [167]:
sample_train_df["answer_word_true_idx"] = sample_train_df.apply(lambda row: get_answer_idx(row["answer_start"], row["answer_word_idx"]), axis=1)

In [168]:
sample_train_df.head()

,title,context,question,answer_start,answer_text,document_id,answer_word_idx,answer_word_true_idx
0,로버트 라이트,"라이트는 종교 주제, 특히 '신'이라는 개념의 역사와 진화에 관해 광범위하게 글을 ...",대중을 통제하기 위해 위정자들의 수단이 되기도 한 것은?,387,종교,57743,"[5, 387, 482, 662, 773, 781, 873, 909, 1038]",1
1,하프라이프의 등장인물 목록,"일라이 밴스(Eli Vance)는 하프라이프와 하프라이프 2에 등장하는 캐릭터로, ...",고든은 블랙메사 연구소의 어디에서 일했나요?,287,C구역,24718,[287],0
2,튀코 브라헤,"튀코 브라헤(Tycho Brahe, 1546년 12월 14일 ~ 1601년 10월 ...",튀코 브라헤는 프레데리크 2세가 사망한 후 어디로 이주하나요?,307,프라하,4944,[307],0
3,동적 평형 상태,"소다로 찬 새로운 병에서, 수용액 상태의 이산화 탄소 농도는 특정 값을 가진다. 만...",농도산수가 일정하기 위한 전제조건은?,589,평형상태,6502,"[247, 589]",1
4,듀이,1938년 듀이는 인기있는 민주당 현직 주지사 허버트 리먼을 상대로 뉴욕 주지사를 ...,듀이가 첫 번째로 뉴욕 주지사가 된 해는?,233,1942년,22802,[233],0


In [170]:
# 괄호 안에 있는 외래어 제거 함수
def remove_other_lang(text):
  return re.sub(r"\(.*?\)", "", text)

In [171]:
sample_train_df["context"] = sample_train_df["context"].apply(remove_other_lang)

In [173]:
sample_train_df.head()

,title,context,question,answer_start,answer_text,document_id,answer_word_idx,answer_word_true_idx
0,로버트 라이트,"라이트는 종교 주제, 특히 '신'이라는 개념의 역사와 진화에 관해 광범위하게 글을 ...",대중을 통제하기 위해 위정자들의 수단이 되기도 한 것은?,387,종교,57743,"[5, 387, 482, 662, 773, 781, 873, 909, 1038]",1
1,하프라이프의 등장인물 목록,"일라이 밴스는 하프라이프와 하프라이프 2에 등장하는 캐릭터로, 고든 프리맨, 아이작...",고든은 블랙메사 연구소의 어디에서 일했나요?,287,C구역,24718,[287],0
2,튀코 브라헤,"튀코 브라헤는 덴마크의 천문학자이다. 로스토크 대학에서 공부했다.\n\n1572년,...",튀코 브라헤는 프레데리크 2세가 사망한 후 어디로 이주하나요?,307,프라하,4944,[307],0
3,동적 평형 상태,"소다로 찬 새로운 병에서, 수용액 상태의 이산화 탄소 농도는 특정 값을 가진다. 만...",농도산수가 일정하기 위한 전제조건은?,589,평형상태,6502,"[247, 589]",1
4,듀이,1938년 듀이는 인기있는 민주당 현직 주지사 허버트 리먼을 상대로 뉴욕 주지사를 ...,듀이가 첫 번째로 뉴욕 주지사가 된 해는?,233,1942년,22802,[233],0


In [174]:
sample_train_df["answer_word_idx_after"] = sample_train_df.apply(lambda row: get_answer_word_idx(row["context"], row["answer_text"]), axis=1)

In [175]:
def get_after_answer_start(index_list, idx):
  if len(index_list) > idx:
    return index_list[idx]
  return 0

In [176]:
sample_train_df["answer_new_start"] = sample_train_df.apply(lambda row: get_after_answer_start(row["answer_word_idx_after"], row["answer_word_true_idx"]), axis=1)

In [177]:
sample_train_df.head()

,title,context,question,answer_start,answer_text,document_id,answer_word_idx,answer_word_true_idx,answer_word_idx_after,answer_new_start
0,로버트 라이트,"라이트는 종교 주제, 특히 '신'이라는 개념의 역사와 진화에 관해 광범위하게 글을 ...",대중을 통제하기 위해 위정자들의 수단이 되기도 한 것은?,387,종교,57743,"[5, 387, 482, 662, 773, 781, 873, 909, 1038]",1,"[5, 387, 482, 662, 773, 781, 873, 909, 1038]",387
1,하프라이프의 등장인물 목록,"일라이 밴스는 하프라이프와 하프라이프 2에 등장하는 캐릭터로, 고든 프리맨, 아이작...",고든은 블랙메사 연구소의 어디에서 일했나요?,287,C구역,24718,[287],0,[276],276
2,튀코 브라헤,"튀코 브라헤는 덴마크의 천문학자이다. 로스토크 대학에서 공부했다.\n\n1572년,...",튀코 브라헤는 프레데리크 2세가 사망한 후 어디로 이주하나요?,307,프라하,4944,[307],0,[263],263
3,동적 평형 상태,"소다로 찬 새로운 병에서, 수용액 상태의 이산화 탄소 농도는 특정 값을 가진다. 만...",농도산수가 일정하기 위한 전제조건은?,589,평형상태,6502,"[247, 589]",1,"[247, 581]",581
4,듀이,1938년 듀이는 인기있는 민주당 현직 주지사 허버트 리먼을 상대로 뉴욕 주지사를 ...,듀이가 첫 번째로 뉴욕 주지사가 된 해는?,233,1942년,22802,[233],0,[210],210


In [178]:
sample_train_df.drop("answer_word_idx", axis=1)

,title,context,question,answer_start,answer_text,document_id,answer_word_true_idx,answer_word_idx_after,answer_new_start
0,로버트 라이트,"라이트는 종교 주제, 특히 '신'이라는 개념의 역사와 진화에 관해 광범위하게 글을 ...",대중을 통제하기 위해 위정자들의 수단이 되기도 한 것은?,387,종교,57743,1,"[5, 387, 482, 662, 773, 781, 873, 909, 1038]",387
1,하프라이프의 등장인물 목록,"일라이 밴스는 하프라이프와 하프라이프 2에 등장하는 캐릭터로, 고든 프리맨, 아이작...",고든은 블랙메사 연구소의 어디에서 일했나요?,287,C구역,24718,0,[276],276
2,튀코 브라헤,"튀코 브라헤는 덴마크의 천문학자이다. 로스토크 대학에서 공부했다.\n\n1572년,...",튀코 브라헤는 프레데리크 2세가 사망한 후 어디로 이주하나요?,307,프라하,4944,0,[263],263
3,동적 평형 상태,"소다로 찬 새로운 병에서, 수용액 상태의 이산화 탄소 농도는 특정 값을 가진다. 만...",농도산수가 일정하기 위한 전제조건은?,589,평형상태,6502,1,"[247, 581]",581
4,듀이,1938년 듀이는 인기있는 민주당 현직 주지사 허버트 리먼을 상대로 뉴욕 주지사를 ...,듀이가 첫 번째로 뉴욕 주지사가 된 해는?,233,1942년,22802,0,[210],210
...,...,...,...,...,...,...,...,...,...
95,운영 체제 시장 점유율,이 항목의 운영 체제 시장 점유율은 특정 시간에 각 운영 체제를 실행하는 컴퓨팅 장...,마이크로소프트사 윈도우의 시장 점유율이 줄어들기 시작한 해는?,406,2016년,21909,0,"[406, 924]",406
96,윤치호,을미사변\n\n1895년7월 윤치호는 남감리교 헨드릭스 감독 에게 조선의 선교를 위...,윤치오가 특파대사직에 고용된 것을 막으려한 나라는?,239,일본,5286,0,"[227, 268, 287, 292, 320, 518, 574]",227
97,도자기,흙을 반죽하여 도자기를 만드는 데는 3종의 성질을 가진 원료가 필요하다.\n* 반죽...,어떤 성분을 많이 넣어야 도자기의 처음 성형했을 때와 구워 냈을때의 형태가 별로 달...,440,석영질의 흙,13784,0,[436],436
98,토레혼데아르도스,토레혼데아르도스는 스페인 마드리드 지방의 기초자치단체이다.\n\n마드리드 시에서 동...,트레혼데아르도스 공군기지의 전투기를 볼 수 있는 곳은?,431,마드리드 바하라스 공항,44105,0,[374],374


In [179]:
def checking_good(df):
  t = 0
  f = 0
  total = len(sample_train_df)
  for i in range(total):
    answer_text = sample_train_df.loc[i]["answer_text"][0]
    answer_start = sample_train_df.loc[i]["answer_new_start"]
    context = sample_train_df.loc[i]["context"]
    try:
      if answer_text == context[answer_start]:
        print(f"True : {answer_text} = {context[answer_start]}")
        t += 1
      else:
        print(f"False : {answer_text} = {context[answer_start]}")
        f += 1
    except:
      continue

  print(f"총 {total}개 중에서 {t}개가 맞고, {f}개가 틀렸습니다.")
  print(f"성공률 : {(t / total) * 100 : .2f}%")

In [ ]:
checking_good(sample_train_df)

In [369]:
# 최종 코드 정리

# pandas datasframe으로 변환 및 필요한 컬럼만 추출하는 함수
def dataset_to_df(dataset):
  data_list = []

  for data in dataset:
    data_list.append({
        'title' : data["title"],
        "context" : data["context"],
        "question" : data["question"],
        "id" : data["id"],
        "answer_start" : data["answers"]["answer_start"][0],
        "answer_text" : data["answers"]["text"][0],
        "document_id" : data["document_id"]
    })
  df = pd.DataFrame(data_list)
  return df

# 괄호 안에 있는 외래어 제거 함수
def remove_other_lang(text):
  return re.sub(r"\([^가-힣0-9]*?\)", "", text)

# 생성한 answer_word_idx에서 answer_start와 같은 값을 가지는 인덱스 번호 저장
def get_answer_idx(answer_start, answer_word_idx):
  re_idx = 0
  for idx, data in enumerate(answer_word_idx):
    if data == answer_start:
      re_idx = int(idx)

  return re_idx

# 바뀐 인덱스 값 리턴해주는 함수
def get_after_answer_start(index_list, idx):
  if len(index_list) > idx:
    return index_list[idx]
  return 0

# answer_text와 같은 단어들의 시작 인덱스를 구하는 코드
def get_answer_word_idx(context, answer):
  indices = []
  index = context.find(answer)

  while index != -1:
    indices.append(index)
    index = context.find(answer, index + 1)

  return indices

# 제대로 변환이 됐는지 확인하는 함수
def checking_good(df):
  t = 0
  f = 0
  total = len(df)
  fail = []

  for i in range(total):
    answer_start = df.loc[i]["answer_start"]
    answer_text = df.loc[i]['answer_text'][0]
    context = df.loc[i]['context'][answer_start]
    if answer_text == context:
      t += 1
    else:
      f += 1
      fail.append(i)
  print(f"총 {total}개 중에서 {t}개가 맞고, {f}개가 틀렸습니다.")
  print(f"성공률 : {(t / total) * 100 : .2f}%")
  if len(fail) > 0:
    print(f"틀린 인덱스 : {fail}")
    return fail
  return []

# 마크 다운 전처리하는 함수
def remove_markdwon(text):
  # 헤더 제거
  text = re.sub(r'^#{1,6}\s', '', text, flags=re.MULTILINE)
  # 볼드체 제거
  text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
  # 이탤릭체 제거
  text = re.sub(r'\*(.*?)\*', r'\1', text)
  # 코드 블록 제거
  text = re.sub(r'```[\s\S]*?```', '', text)
  # 인라인 코드 제거
  text = re.sub(r'`([^`\n]+)`', r'\1', text)
  # 링크 제거 (텍스트만 유지)
  text = re.sub(r'\[(.*?)\]\(.*?\)', r'\1', text)
  # 리스트 기호 제거
  text = re.sub(r'^\s*[-*+]\s', '', text, flags=re.MULTILINE)
  return text.strip()

# answer들을 원래의 형태로 묶어주는 함수
def combine_answer(df):
  return df[['answer_start', 'answer_text']].apply(lambda x: {'answer_start': [(x['answer_start'])], 'text': [x['answer_text']]}, axis=1)

# 개행 제거하는 함수
def clean_text(text):
  # 개행 문자 제거
  text = re.sub(r'\n|\\n', ' ', text)
  # 연속된 공백 제거
  text = re.sub(r'\s+', ' ', text)
  return text.strip()

def main(df, newline=False, mark=False, lang=True):
  # 작업 전 성능 테스트
  print("Before =======")
  checking_good(df)

  # answer_word_idx라는 컬럼에 정답과 같은 단어들이 시작하는 인덱스 값을 저장
  df["answer_word_idx"] = df.apply(lambda row: get_answer_word_idx(row["context"], row["answer_text"]), axis=1)

  # answer_word_true_idx라는 컬럼에 answer_word_idx를 기준으로 정답 value의 인덱스를 저장
  df["answer_word_true_idx"] = df.apply(lambda row: get_answer_idx(row["answer_start"], row["answer_word_idx"]), axis=1)

  # context에 외래어 제거해서 다시 context에 저장 -- 이 위치에 다른 전처리 작업 수행 가능
  if newline:
    df["context"] = df["context"].apply(clean_text)
  if mark:
    df["context"] = df["context"].apply(remove_markdwon)
  if lang:
    df["context"] = df["context"].apply(remove_other_lang)
    df["answer_text"] = df["answer_text"].apply(remove_other_lang)

  # 전처리 후에 바뀐 context에서 정답과 같은 단어들이 시작하는 인덱스 값을 저장
  df["answer_word_idx_after"] = df.apply(lambda row: get_answer_word_idx(row["context"], row["answer_text"]), axis=1)

  # answer_word_idx_after에서 answer_word_true_idx번재의 인덱스 값을 answer_start에 덮어서 저장
  df["answer_start"] = df.apply(lambda row: get_after_answer_start(row["answer_word_idx_after"], row["answer_word_true_idx"]), axis=1)

  # 작업 후 성능 테스트
  print("\nAfter =======")
  fails = checking_good(df)
  if len(fails) > 0:
    print("\n잘 안된 인덱스")
    for fail in fails:
      print(df.loc[fail]["answer_text"])
      print(df.loc[fail]["answer_start"])
      print(df.loc[fail]["context"])
      print(df.loc[fail]["question"])
      print("===============")

  # 필요 없는 컬럼 삭제
  df = df.drop(columns=["answer_word_idx", "answer_word_true_idx", "answer_word_idx_after"])

  return df

In [182]:
path = "your_train_dataset_path"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])

In [183]:
main(train_df)

Before =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%


,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 ...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,213,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,504,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,611,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334
...,...,...,...,...,...,...
3947,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,317,지질 구조,8275
3948,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,616,뉴저지,37764
3949,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,255,칼 에케르트,12730
3950,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,476,슬라브 신화,10924


# 결론
- 괄호 안에 외래어만 존재하는 경우는 모두 제거. 한글이나 숫자가 같이 존재하는 경우에는 제거되지 않음.

# 외래어 제거 파일 생성

In [184]:
train_path = "your_train_dataset_path"
train_dataset = load_from_disk(train_path)

In [221]:
train_df = dataset_to_df(train_dataset["train"])
valid_df = dataset_to_df(train_dataset["validation"])

In [186]:
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [222]:
# 외래어 제거 전처리
train_df = main(train_df)

Before =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%


In [188]:
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 ...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,213,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,504,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,611,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [223]:
# answer_start와 answer_text 합치는 코드
train_df["answers"] = train_df.apply(lambda row: {'answer_start': row['answer_start'], 'text': row['answer_text']}, axis=1)

train_df = train_df.drop(columns=['answer_start', 'answer_text'])

In [224]:
train_df.head()

,title,context,question,document_id,answers
0,미국 상원,미국 상의원 또는 미국 상원은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 ...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,18293,"{'answer_start': 213, 'text': '하원'}"
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,51638,"{'answer_start': 212, 'text': '《경영의 실제》'}"
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,5028,"{'answer_start': 504, 'text': '백성'}"
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,34146,"{'answer_start': 611, 'text': '중국'}"
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,47334,"{'answer_start': 30, 'text': '4개'}"


In [225]:
valid_df = main(valid_df)

Before =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%


In [226]:
# answer_start와 answer_text 합치는 코드
valid_df["answers"] = valid_df.apply(lambda row: {'answer_start': row['answer_start'], 'text': row['answer_text']}, axis=1)

valid_df = valid_df.drop(columns=['answer_start', 'answer_text'])

In [227]:
valid_df.head()

,title,context,question,document_id,answers
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,9027,"{'answer_start': 284, 'text': '한보철강'}"
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,51765,"{'answer_start': 146, 'text': '1871년'}"
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,15694,"{'answer_start': 517, 'text': '나뭇잎'}"
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,59536,"{'answer_start': 1044, 'text': '금대야'}"
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,32991,"{'answer_start': 386, 'text': '수평적 관계'}"


In [192]:
!pip install pyarrow

In [320]:
path = "your_train_dataset_path"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])
valid_df = dataset_to_df(dataset["validation"])

## 외래어 전처리 dataset

In [346]:
# 외래어만 제거

# 데이터 프레임을 바로 Dataset으로 변환
path = "your_train_dataset_path"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])
valid_df = dataset_to_df(dataset["validation"])

train_df = main(train_df)
valid_df = main(valid_df)

train_df['answers'] = combine_answer(train_df)
train_df = train_df.drop(columns=['answer_start', 'answer_text'])
valid_df['answers'] = combine_answer(valid_df)
valid_df = valid_df.drop(columns=['answer_start', 'answer_text'])

save_path = "your_path/train_remove_lang_dataset"

dataset_train = Dataset.from_pandas(train_df)
dataset_valid = Dataset.from_pandas(valid_df)

# DatasetDict 생성
dataset_dict = DatasetDict({
    'train': dataset_train,
    'validation': dataset_valid
})

dataset_dict.save_to_disk(save_path)

#dataset_train.save_to_disk(f"{save_path}/train")
#dataset_valid.save_to_disk(f"{save_path}/validation")

Before =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%
Before =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%


Saving the dataset (0/1 shards):   0%|          | 0/3952 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/240 [00:00<?, ? examples/s]

# 개행, 마크다운, 외래어 전처리 dataset

In [348]:
# 개행, 마크다운 전처리 및 외래어 제거 및 answer_start 위치 수정

path = "your_train_dataset_path"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])
valid_df = dataset_to_df(dataset["validation"])

train_df = main(train_df, newline=True, mark=True)
valid_df = main(valid_df, newline=True, mark=True)

# answer_start와 answer_text 합치는 코드
train_df['answers'] = combine_answer(train_df)
train = train_df.drop(columns=['answer_start', 'answer_text'])
valid_df['answers'] = combine_answer(valid_df)
valid = valid_df.drop(columns=['answer_start', 'answer_text'])

# 데이터 프레임을 Dataset으로 저장
save_path = "your_path/train_remove_lang_newline_markdown_dataset"

dataset_train = Dataset.from_pandas(train_df)
dataset_valid = Dataset.from_pandas(valid_df)

# DatasetDict 생성
dataset_dict = DatasetDict({
    'train': dataset_train,
    'validation': dataset_valid
})

dataset_dict.save_to_disk(save_path)

Before =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%
Before =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%


Saving the dataset (0/1 shards):   0%|          | 0/3952 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/240 [00:00<?, ? examples/s]

In [272]:
check_newline(train)

\n의 개수 : 0


In [273]:
check_newline(valid)

\n의 개수 : 0


In [357]:
path = "your_path/train_remove_lang_newline_markdown_dataset"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])
valid_df = dataset_to_df(dataset["validation"])

## 개행 전처리 dataset

In [376]:
# 개행 제거 dataset

# 데이터 프레임을 바로 Dataset으로 변환
path = "your_train_dataset_path"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])
valid_df = dataset_to_df(dataset["validation"])

train_df = main(train_df, newline=True, lang=False)
valid_df = main(valid_df, newline=True, lang=False)

train_df['answers'] = combine_answer(train_df)
train_df = train_df.drop(columns=['answer_start', 'answer_text'])
valid_df['answers'] = combine_answer(valid_df)
valid_df = valid_df.drop(columns=['answer_start', 'answer_text'])

save_path = "your_path/train_remove_newline_dataset"

dataset_train = Dataset.from_pandas(train_df)
dataset_valid = Dataset.from_pandas(valid_df)

# DatasetDict 생성
dataset_dict = DatasetDict({
    'train': dataset_train,
    'validation': dataset_valid
})

dataset_dict.save_to_disk(save_path)

Before =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%
Before =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%


Saving the dataset (0/1 shards):   0%|          | 0/3952 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/240 [00:00<?, ? examples/s]

## 마크다운 전처리 dataset

In [377]:
# 마크다운 제거 dataset

# 데이터 프레임을 바로 Dataset으로 변환
path = "your_train_dataset_path"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])
valid_df = dataset_to_df(dataset["validation"])

train_df = main(train_df, mark=True, lang=False)
valid_df = main(valid_df, mark=True, lang=False)

train_df['answers'] = combine_answer(train_df)
train_df = train_df.drop(columns=['answer_start', 'answer_text'])
valid_df['answers'] = combine_answer(valid_df)
valid_df = valid_df.drop(columns=['answer_start', 'answer_text'])

save_path = "your_path/train_remove_markdown_dataset"

dataset_train = Dataset.from_pandas(train_df)
dataset_valid = Dataset.from_pandas(valid_df)

# DatasetDict 생성
dataset_dict = DatasetDict({
    'train': dataset_train,
    'validation': dataset_valid
})

dataset_dict.save_to_disk(save_path)

Before =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%
Before =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%


Saving the dataset (0/1 shards):   0%|          | 0/3952 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/240 [00:00<?, ? examples/s]

## 개행, 마크다운 전처리 dataset

In [378]:
# 개행, 마크다운 제거 dataset

# 데이터 프레임을 바로 Dataset으로 변환
path = "your_train_dataset_path"
dataset = load_from_disk(path)
train_df = dataset_to_df(dataset["train"])
valid_df = dataset_to_df(dataset["validation"])

train_df = main(train_df, newline=True, mark=True, lang=False)
valid_df = main(valid_df, newline=True, mark=True, lang=False)

train_df['answers'] = combine_answer(train_df)
train_df = train_df.drop(columns=['answer_start', 'answer_text'])
valid_df['answers'] = combine_answer(valid_df)
valid_df = valid_df.drop(columns=['answer_start', 'answer_text'])

save_path = "your_path/train_remove_newline_markdown_dataset"

dataset_train = Dataset.from_pandas(train_df)
dataset_valid = Dataset.from_pandas(valid_df)

# DatasetDict 생성
dataset_dict = DatasetDict({
    'train': dataset_train,
    'validation': dataset_valid
})

dataset_dict.save_to_disk(save_path)

Before =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 3952개 중에서 3952개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%
Before =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%

After =======
총 240개 중에서 240개가 맞고, 0개가 틀렸습니다.
성공률 :  100.00%


Saving the dataset (0/1 shards):   0%|          | 0/3952 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/240 [00:00<?, ? examples/s]